# Freespace Garment Simulation

Notebook for simulating garments moving in freespace. This will ultimately be used to generate training data for the tracking model we're putting together.

**Note:** We probably want to have two modes:
- Simple dynamics where the cloth behaves like a quasi-rigid object.
  - This should be easy-peasy for the network to track since it's essentially a rigid body dynamics problem if the gripper is moving slowly enough.
- More complicated dynamics where the cloth deforms due to the "gripper" moving faster than the cloth can reach a quasi-static state.
  - This will be harder for the network to predict but it should still be possible.

Ultimately we probably won't want hard distinctions between the two modes of operation but splitting them like this will help me to develop the functions/classes necessary to write the data generation pipeline.

## TODO

- ~~Set animation interpolation scheme to 'linear' to simplify things here.~~
  - ~~Seems like you can do `keyframe_variable.interpolation = "LINEAR"`~~
  - Actually, smooth motion is probably preferable. Linear is too sharp.
    - Can maybe edit F-Curve properties to make it sharper if necessary?

In [14]:
import os
import sys
from pathlib import Path

import bpy
import mathutils
import numpy as np

# Imports from this repository
sys.path.append("../../")
sys.path.append("../")
from simulation.cloth_3d_util.accessor import Cloth3DCanonicalAccessor
from simulation.cloth_3d_util.util import loadInfo
from simulation.pipeline.simulate_garment_hanging_rest_state import \
    simulate_garment_hanging_rest_state
from simulation.blender_util_dylan.physics import set_sim_output_as_default_mesh_shape
from simulation.blender_util_dylan.checkpointer import BlendFileCheckpointer
from simulation.blender_util_dylan.gripper import GripperAnimation
from simulation.blender_util_dylan.debug import print_obj_keyframe_coordinates

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
FILE_ROOT = Path(os.getcwd())
CLOTH3D_PATH = Path(os.path.expanduser("~/DataLocker/datasets/CLOTH3D/training/"))
OUTPUT_ROOT = FILE_ROOT / ".." / "script_output" / "sim_pipeline_driver_test"

# Make the output directory if it doesn't exist.
OUTPUT_ROOT.mkdir(exist_ok=True)


PLANE_OFFSET = 0.025  # [m]

In [4]:
sample_configuration = {
    "sample_id": "00380",
    "garment_name": "Tshirt",
    "grip_vertex_idx": 0
}

smpl_simulation_duration_pair = (0, 120)

sample_configs = [sample_configuration]

## Start With Simple Freespace Dynamics

This is where the garment behaves much like a rigid body during motion, meaning the gripper will have to move fairly slowly.

I think this should be packaged up to where we have a routine for simulating one freespace motion in one 3D vector direction. Doing anything more complicated would introduce the possibility of complicated dynamics.

I just baked a garment simulation in Blender and moving in direction $[0, 0, -1]^T$ over 100 frames with a velocities:
- $0.0016 / \text{frame}$ was fine.
- $0.0032 / \text{frame}$ was fine.
- $0.0064 / \text{frame}$ looked okay. I suspect that if it was in the $X$ or $Y$ direction it would be less okay.

Moving in $[1, 0, 0]^T$ direction.
- $0.0064 / \text{frame}$ looked fine even with linear interpolation.

In [5]:
# Example of what the freespace dynamics arguments could look like.
# grip_lowering_args = {
#     "initial_sim_end_frame": smpl_simulation_duration_pair[1],
#     "start_frame": 1,
#     "end_frame": 100,
#     "fraction_lowered": 0.25, # Fraction of the cloth that will be lowered onto the table.
# }

# class FreespaceSimpleSimArgs:
#     def __init__(self, direction_vec: np.ndarray, velocity: float, )

# freespace_sim_args = {
#     "direction": np.array((1.0, 0.0, 0.0))
# }



### Dynamics Checkpointing: Simulate Or Load Hanging Resting State

The point of checkpointing is so that we don't have to resimulate the grasped resting state every time we want to run a new dynamics simulation on the same garment.

In [6]:
## Setup the simulation.
accessor = Cloth3DCanonicalAccessor(CLOTH3D_PATH)
config = sample_configs[0]
sample_key = f"{config['sample_id']}_{config['garment_name']}_{config['grip_vertex_idx']}_simple_dynamics"
sample_dir = OUTPUT_ROOT / sample_key
sample_dir.mkdir(exist_ok=True)

result_file = "simulation_result.pk"
result_path = sample_dir / result_file

# Get a dictionary containing the data for this sample garment.
sample_data = accessor.get_sample_data(**config)

garment_info_mat_filename = CLOTH3D_PATH / config["sample_id"] / "info.mat"
garment_info = loadInfo(garment_info_mat_filename)

checkpointer = BlendFileCheckpointer(sample_dir, save_new_checkpoints=True)

In [7]:
# Confirmed to work as expected!
if checkpointer.does_rest_state_checkpoint_exist():
    print("Hanging state checkpoint exists. Loading checkpoint from file.")
    # Load the checkpoint instead of simulating the resting state again.
    checkpointer.load_hanging_rest_state()
else:
    print("Hanging state checkpoint does not exist. Simulating and saving checkpoint.")
    # Run the SMPL simulation to get the garment in a stationary hanging configuration
    frames_to_resting_state = 120
    result_data_smpl = simulate_garment_hanging_rest_state(config, sample_data, frames_to_resting_state)

    # Then set the default mesh shape to this shape
    cloth_obj = bpy.data.objects["cloth"]
    set_sim_output_as_default_mesh_shape(cloth_obj, frames_to_resting_state)

    # Now create a checkpoint for this so we don't have to resimulate every time we run this
    # notebook.
    checkpointer.save_hanging_rest_state()

Hanging state checkpoint exists. Loading checkpoint from file.
Read blend: /home/dcolli23/code/school/rob599_deeprob/projects/final/garmentnets_tracking/simulation/runners/../script_output/sim_pipeline_driver_test/00380_Tshirt_0_simple_dynamics/hanging_rest_state.blend
Successfully reloaded hanging rest state checkpoint.


## Planning Dynamics Routines

Now that we've simulated the garment hanging rest state and set that as the default mesh shape, we can start doing dynamics runs.

We will control the motion of the "gripper" (empty object that the cloth vertices are pinned to) by controlling the keyframes - the trajectory points - and the F-curves - the interpolation method between keyframes.

For Blender, the following is required for the definitions:
- Each keypoint needs to be defined by:
    - Object origin location
    - Frame with which to insert the keyframe
- F-curves - default is Bezier with pretty reasonable parametrs
  - For linear control, no parameters are needed.
  - For other interpolation methods I'm unsure.

**TODO:** Decide on Fcurve interpolation scheme (linear or not?). Non-linear seems preferable?
- Let's just use the default F-Curves and get as far as we can with this setting.
**TODO:** Decide on programmatic representation. Velocity control with a unit direction vector seems reasonable.

For keyframe insertion, can potentially follow this pattern:
```python
obj = bpy.context.active_object  # Or however else we get the object pointer
for keyframe_frame, obj_location in (keyframes):
    obj.location = obj_location  # I believe this can be a flat numpy array
    obj.keyframe_insert(data_path="location",
                        index=-1,  # indicates to place the keyframe last
                        group=group,  # group name F-curve added to if it doesn't exist
                        frame=keyframe_frame)
```

In [8]:
cloth_obj = bpy.data.objects["cloth"]
gripper_obj = bpy.data.objects["Empty"]

In [9]:
gripper_animation = GripperAnimation(gripper_obj, cloth_obj)

Starting gripper keyframe frame: 0


In [11]:
using_simple_dynamics = False

velocity_mps = 0.15 if using_simple_dynamics else 0.75

direction_vecs = [
    np.array((1.0, 0.0, 0.0)),
    np.array((0.0, -1.0, 0.0)),
    np.array((0.0, 0.0, -1.0)),
    np.array((-1.0, -1.0, 1.0))
]

frame_duration = 50

for vec in direction_vecs:
    gripper_animation.add_movement(vec, velocity_mps, frame_duration)

# Now bake the sim.
bpy.ops.ptcache.bake_all()

bake: frame 0 :: 200
bake: frame 1 :: 200
bake: frame 2 :: 200
bake: frame 3 :: 200
bake: frame 4 :: 200
bake: frame 5 :: 200
bake: frame 6 :: 200
bake: frame 7 :: 200
bake: frame 8 :: 200
bake: frame 9 :: 200
bake: frame 10 :: 200
bake: frame 11 :: 200
bake: frame 12 :: 200
bake: frame 13 :: 200
bake: frame 14 :: 200
bake: frame 15 :: 200
bake: frame 16 :: 200
bake: frame 17 :: 200
bake: frame 18 :: 200
bake: frame 19 :: 200
bake: frame 20 :: 200
bake: frame 21 :: 200
bake: frame 22 :: 200
bake: frame 23 :: 200
bake: frame 24 :: 200
bake: frame 25 :: 200
bake: frame 26 :: 200
bake: frame 27 :: 200
bake: frame 28 :: 200
bake: frame 29 :: 200
bake: frame 30 :: 200
bake: frame 31 :: 200
bake: frame 32 :: 200
bake: frame 33 :: 200
bake: frame 34 :: 200
bake: frame 35 :: 200
bake: frame 36 :: 200
bake: frame 37 :: 200
bake: frame 38 :: 200
bake: frame 39 :: 200
bake: frame 40 :: 200
bake: frame 41 :: 200
bake: frame 42 :: 200
bake: frame 43 :: 200
bake: frame 44 :: 200
bake: frame 45 :: 20

{'FINISHED'}

In [15]:
print_obj_keyframe_coordinates(gripper_obj)

Axis: x
	Frames: [  0.  50. 100. 150. 200.]
	Coordinates: [3.725290298461914e-09, 1.5625, 1.5625, 1.5625, 0.0]
Axis: y
	Frames: [  0.  50. 100. 150. 200.]
	Coordinates: [0.0, 0.0, -1.5625, -1.5625, -3.125]
Axis: z
	Frames: [  0.  50. 100. 150. 200.]
	Coordinates: [-1.4901161193847656e-08, -1.4901161193847656e-08, -1.4901161193847656e-08, -1.5625, 0.0]


In [ ]:
bpy.ops.wm.save_as_mainfile(filepath=(sample_dir / "gripper_animation_check.blend").as_posix())

Info: Saved "gripper_animation_check.blend"


{'FINISHED'}

## Complicated Dynamics

I initially thought doing the complicated dynamics would be tougher. Instead, I think I can animate curves by just keyframing select axes instead of all x, y, z at every keyframe.

**TODO:** If deemed necessary, add functionality in the `GripperAnimation` class to accept which axes to keyframe. I should be able to do this by just accepting the axes index or the axis key (e.g. 'x') and using a map from key -> index. Seems like the mapping is straightforward (i.e. {'x': 0, 'y': 1, 'z': 2}).

I can also induce deformation by moving the gripper in a way that changes direction quickly.
**Note:** I imagine the *impulse* is what's important here. Quickly changing the velocity of the gripper.
- Due to this, I might have to change the Fcurve for the "gripper" motion to linear so that it quickly jerks the cloth instead of the typical smooth interpolation scheme.